## 패션 이미지 데이터셋 전처리

In [1]:
# import pandas as pd

# path = 'data/meta/'
# train = pd.read_json(path + 'train_no_dup.json')
# valid = pd.read_json(path + 'valid_no_dup.json')
# test = pd.read_json(path + 'test_no_dup.json')
# blank = pd.read_json(path + 'fill_in_blank_test.json')

## Data Load Example

In [26]:
import pandas as pd
from dataloader.sampling import DataSampler
#from dataloader.multimodal_data import MultiModalData
from dataloader.multilstm_data import MultiModalData
import os

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
meta_dir = os.path.join(data_dir, 'meta')
image_dir = os.path.join(data_dir, 'images')
sampler = DataSampler(data_path = meta_dir, k=50, test_sampling_ratio=1)
concat_df, question_data = sampler.sample_data()

train_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='train')
valid_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='valid')
test_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, question = question_data, mode='test')

In [37]:
concat_df.head(10)

,name,items,desc,set_id,type
0,School,"[{'index': 1, 'name': 'colors crop top tee', '...",#red #outfit #beautiful #converse #geek #school,100119331,train
1,Fashion Polyvore!,"[{'index': 1, 'name': 'shein sheinside black l...",A fashion look from October 2014 featuring cot...,136372956,train
2,red & pink,"[{'index': 1, 'name': 'maya maxi dress', 'pric...",A fashion look from June 2016 by lisamichele-c...,200277215,train
3,Cool chic,"[{'index': 1, 'name': 'black short sleeve lace...",http://www.romwe.com/Black-Faux-Suede-Button-F...,206996085,train
4,Different formal style,"[{'index': 1, 'name': 'james contrast panel sh...",A fashion look from November 2014 featuring co...,141781954,train
5,Perfect Puffer Jackets,"[{'index': 1, 'name': 'mcq alexander mcqueen w...",#puffers,214379138,train
6,Sweet Jeans!,"[{'index': 1, 'name': 'rails long sleeve arrow...",A fashion look from January 2017 by agnesmakon...,214877329,train
7,School party with Leo,"[{'index': 1, 'name': 'young wild free iphone ...",A beauty collage from January 2014 featuring e...,110637600,train
8,Three Piece Outfit,"[{'index': 1, 'name': 'boutique moschino cardi...",A fashion look from September 2016 by sjlew fe...,207266399,train
9,Date night with Luke,"[{'index': 1, 'name': 'light blue lace belted ...",A fashion look from February 2016 by joanaoliv...,189795216,train


In [10]:
train_dataset[10].keys()

dict_keys(['texts', 'images', 'set_id'])

In [39]:
train_dataset[5]

{'texts': tensor([[ 0.1392, -0.0566,  0.0559,  ...,  0.2035,  0.2490,  0.2822],
         [ 0.0786,  0.3379,  0.2666,  ...,  0.2688,  0.1572, -0.1230],
         [ 0.3015,  0.1747, -0.2034,  ...,  0.2964, -0.0151, -0.0194],
         [ 0.1310, -0.1548,  0.1309,  ...,  0.4358,  0.0953, -0.0227],
         [ 0.4648,  0.0629, -0.4102,  ...,  0.8442,  0.1186,  0.4973],
         [-0.0402,  0.0879, -0.1364,  ...,  0.3044,  0.3669,  0.1902]],
        device='cuda:0', grad_fn=<ToCopyBackward0>),
 'images': tensor([[ 0.0453, -0.2100,  0.2167,  ...,  0.5605, -0.0876,  0.6680],
         [-0.1577,  0.3599,  0.1707,  ...,  0.4045,  0.0636,  0.6235],
         [-0.0429,  0.3560,  0.3254,  ...,  0.8545, -0.0536,  0.0138],
         [-0.1941, -0.2360,  0.6460,  ...,  0.6987,  0.2035,  0.1857],
         [ 0.0328, -0.1943, -0.1786,  ...,  0.4023,  0.1215,  0.3862],
         [ 0.2908,  0.3110, -0.3909,  ...,  0.7476,  0.1606,  0.2140]],
        device='cuda:0', grad_fn=<ToCopyBackward0>),
 'set_id': 214379138}

In [33]:
for i in range(len(train_dataset)):
    print(train_dataset[i]['images'])

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [28]:
len(train_dataset), len(valid_dataset)

(1749, 305)

## Dataloader

In [30]:
from torch.nn.utils.rnn import pad_sequence
import torch
def collate_fn(batch):
    """
    배치 내 데이터를 패딩 처리하고 필요한 키로 구성된 dictionary 반환.
    """
    image_embeddings = [item['images'] for item in batch]
    seq_embeddings = [item['texts'] for item in batch]
    lengths = torch.tensor([len(seq) for seq in image_embeddings])

    image_embeddings_padded = pad_sequence(image_embeddings, batch_first=True)
    seq_embeddings_padded = pad_sequence(seq_embeddings, batch_first=True)
    mask = (image_embeddings_padded.sum(dim=2) != 0).float()

    return {
        'image_embeddings': image_embeddings_padded,
        'seq_embeddings': seq_embeddings_padded,
        'lengths': lengths,
        'mask': mask
    }

In [31]:
from torch.utils.data import Dataset, DataLoader
#from AI_MMR.model.multilstm import Multifusion, MultiLSTM
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train_dataloader = DataLoader(
        train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
    )
valid_dataloader = DataLoader(
        valid_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn
    )

In [21]:
# Dataloader
 
for batch_idx, batch in enumerate(train_dataloader):
    print(batch.shape)
    break

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
 
for batch_idx, batch in enumerate(valid_dataloader):
    print(batch.shape)
    break

In [4]:
mlp = Multifusion().to(device)
lstm = MultiLSTM(input_size=512, hidden_size=512, num_layers=1, bidirectional=True).to(device)

In [24]:
from AI_MMR.model.multilstm import Multifusion